In [ ]:
from collections import deque
import numpy as np
import glob

"""
Zvolil jsem průchod BFS (Breadth-First Search) pro hledání cesty v bludišti, protože
BFS je vhodný pro hledání nejkratší cesty v neorientovaných grafech a mřížkách.
Zároveň jsme BFS implementovali v předmětu Algoritmy 1 a vím o něm více 
než o Dijkstrově algoritmu...
"""

# Předpokládám, že tento kód bude spuštěn v adresáři, kde se nachází pouze CSV soubor s maticí
# Najde všechny CSV soubory v aktuálním adresáři 
# (dopředu nevím, jaký bude název csv souboru, proto to řeším takto)
csv_files = glob.glob("*.csv")

if csv_files:
    file_path = csv_files[0] # vezme první nalezený CSV soubor
    data = np.loadtxt(file_path, delimiter=",")
    # vznikne binary matrix: 0 → True, 1 → False
    binary_matrix = (data == 0)

else:
    print("Nebyl nalezen žádný CSV soubor.")
    exit()



def save_path_to_csv(path_map, filename="vystup.csv"):
    # True → 0, False → 1
    output_matrix = np.where(path_map, 0, 1)
    
    # Uloží do CSV se separátorem ","
    np.savetxt(filename, output_matrix, fmt="%d", delimiter=",")
    print(f"Výstupní matice byla uložena jako '{filename}'.")


def get_neighbors(i, j, n):
    neighbors = []
    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # nahoru, dolů, vlevo, vpravo

    for di, dj in directions:
        ni, nj = i + di, j + dj
        if 0 <= ni < n and 0 <= nj < n: # zkontrolujeme, jestli je potenciální soused v rámci matice 
            neighbors.append((ni, nj))  # přidáme souseda jako dvojici (řádek, sloupec)

    return neighbors


def show_path(n, parent_map, end):
    path_map = np.full((n, n), False, dtype=bool)
    # vytvoříme matici pro zobrazení cesty, kde True znamená, že buňka je součástí cesty
    current = end
    while current is not None:
        path_map[current] = True
        current = parent_map[current]

    return path_map



def solve(matrix):
    n = matrix.shape[0]
    state_map = np.full(matrix.shape, "unknown", dtype=object) 
    # počáteční stav všech buněk je "unknown", v průběhu prohledávání se mění na "discovered" a "finished"
    parent_map = np.full(matrix.shape, None, dtype=object)
    # parent_map slouží k uchování "předků", abychom mohli sledovat cestu zpět.
    # pro konkrétní buňky bude obsahovat dvojice (i, j) pro jejich předka.
    # dtype=object -> NumPy musí vědět, že ty položky jsou obecné Python objekty.
    # položky jsou buď dvojice (i, j) nebo None (pro počáteční buňku) pro parent_map
    # a "unknown", "discovered", "finished" pro state_map
    queue = deque()

    state_map[0, 0] = "discovered"
    parent_map[0, 0] = None # počáteční buňka nemá žádného předka
    queue.append((0, 0))  # algoritmus vždy začneme v levém horním rohu

    while queue:
        # klasický bsf algoritmus
        i, j = queue.popleft()
        
        for ni, nj in get_neighbors(i, j, n):
            if (ni, nj) == (n-1, n-1):
                # pokud jsme dosáhli pravého dolního rohu, můžeme skončit
                parent_map[ni, nj] = (i, j)
                path_map = show_path(n, parent_map, (n-1, n-1))
                return path_map
            if state_map[ni, nj] == "unknown" and matrix[ni, nj]:
                state_map[ni, nj] = "discovered"
                parent_map[ni, nj] = (i, j)
                queue.append((ni, nj))
        state_map[i, j] = "finished"

    # Pokud se sem dostaneme, žádná cesta neexistuje
    print("Cesta nebyla nalezena.")
    return None


solve_path = solve(binary_matrix)

if solve_path is not None:
    save_path_to_csv(solve_path)
    # Pokud je cesta nalezena, uložíme ji do CSV souboru









Výstupní matice byla uložena jako 'vystup.csv'.
